In [1]:
from GAN import Generator, Discriminator
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt

from Signal_Generator import *
from Signal_Analyzer import *

In [2]:
dataset = []

for i in range(10):
    SG = Signal_Generator(num_sources=1, noise_amplitude=1)
    signals = SG.generating_signal()
    params = SG.printing_parameters()
    signal = signals['Signal'].values

    signal_tensor = torch.tensor(signal, dtype=torch.float).unsqueeze(0).unsqueeze(0)
    params_tensor = torch.tensor(params, dtype=torch.float).unsqueeze(0)

    dataset.append((signal_tensor, params_tensor))

num_latent_variables = 10
z = torch.randn(1, num_latent_variables, 1)

In [3]:
generator = Generator(in_channels=1, num_latent_variables=num_latent_variables, length=len(signal), num_parameters=len(params))
discriminator = Discriminator(input_channels=1, num_latent_variables=num_latent_variables, length=len(signal), num_parameters=len(params))

In [4]:
criterion = nn.BCELoss()

learning_rate = 0.0001
num_epochs = 10

optimizer_G = torch.optim.Adam(generator.parameters(), lr=learning_rate)
optimizer_D = torch.optim.Adam(discriminator.parameters(), lr=learning_rate)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [5]:
for epoch in range(num_epochs):
    for i,data in enumerate(dataset):
        signal_tensor, params_tensor = data
        
        # Generate Fake Parameters
        fake_params = generator(signal_tensor, z).detach().squeeze().numpy()
        fake_params_tensor = torch.tensor(fake_params, dtype=torch.float).unsqueeze(0)
        
        # Train Discriminator
        discriminator.zero_grad()

        real_output = discriminator(signal_tensor, params_tensor, z)
        fake_output = discriminator(signal_tensor, fake_params_tensor, z)

        loss_D = criterion(real_output, torch.ones_like(real_output)) + criterion(fake_output, torch.zeros_like(fake_output))
        loss_D.backward()
        optimizer_D.step()
        
        # Train Generator
        generator.zero_grad()
        fake_output = discriminator(signal_tensor, fake_params_tensor, z)
        loss_G = criterion(fake_output, torch.ones_like(fake_output))
        loss_G.backward()
        optimizer_G.step()
        
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss_D: {loss_D.item():.4f}, Loss_G: {loss_G.item():.4f}")

Epoch [1/10], Loss_D: 1.6543, Loss_G: 0.4006
Epoch [2/10], Loss_D: 1.3948, Loss_G: 0.7393
Epoch [3/10], Loss_D: 1.3911, Loss_G: 0.6292
Epoch [4/10], Loss_D: 1.3862, Loss_G: 0.7081
Epoch [5/10], Loss_D: 1.3869, Loss_G: 0.7187
Epoch [6/10], Loss_D: 1.3862, Loss_G: 0.6854
Epoch [7/10], Loss_D: 1.3861, Loss_G: 0.6923
Epoch [8/10], Loss_D: 1.3862, Loss_G: 0.7026
Epoch [9/10], Loss_D: 1.3861, Loss_G: 0.6960
Epoch [10/10], Loss_D: 1.3860, Loss_G: 0.6959


In [6]:
SG = Signal_Generator(num_sources=1, noise_amplitude=1)
signals = SG.generating_signal()
params = SG.printing_parameters()
signal = signals['Signal'].values

print(params)

signal_tensor = torch.tensor(signal, dtype=torch.float).unsqueeze(0).unsqueeze(0)
params_tensor = torch.tensor(params, dtype=torch.float).unsqueeze(0)

generate_params = generator(signal_tensor, z).detach().squeeze().numpy()
print(generate_params)

[10.43319273366852, 10.259742529573291, 10.959173513768146, 10.676921137759432, 9.809036577895048, 8.581171554357322, 7.613501375017421, 8.98594306841666, 7.832866078212174, 9.240498889825918, 0.30199067229230225, 0.36430767297208155, 0.37286348788763474, 0.27875175740814073, 0.20282325558438544, 0.14605874964494983, 0.3873417106788896, 0.4190080190712868, 0.37610490965111726, 0.1544722450698362, 1.8606030624609833, 0.0635619970403771, 4.52135479767825, 1.1299943002069925, 2.0307257934070457, 2.2823583017062026, 1.383340323003153, 4.319532863326387, 5.147604728342666, 0.6904155822801367]
[ 0.10253743 -0.36689728 -0.11562443]


In [7]:
len(params)

30

In [8]:
len(generate_params)

3